In [ ]:
import numpy as np
import pandas as pd
from plyfile import PlyData
import taichi as ti
import os
import taichi.math as fuck

# 初始化Taichi
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
ti.init(arch=ti.cuda)
res = 0.0002
dims = (400, 520, 200)
grid_points = np.prod(dims)
grid_location = np.zeros((grid_points, 3), dtype=int)
grid_intensity = np.zeros((grid_points,), dtype=float)

# 填充grid_location

@ti.kernel
def init_grid_location(
    location:ti.types.ndarray(),
):
    for idx in ti.ndrange(41600000):
        i = (idx%(400*520))%400
        j = (idx%(400*520))/400
        k = idx/(400*520)
        location[idx,0] = i
        location[idx,1] = j
        location[idx,2] = k

init_grid_location(grid_location)
grid_location = grid_location


In [2]:
# 读入点云文件
ply_path = 'point_cloud_fine/ball_160_after.ply'
plydata = PlyData.read(ply_path)

# 提取点云数据到相应的数组
cloud_location = np.zeros((plydata['vertex'].count, 3))
source_p0 = np.zeros((plydata['vertex'].count,))
radius = np.zeros((plydata['vertex'].count,))

for i, vertex in enumerate(plydata['vertex']):
    cloud_location[i] = [vertex['x'], vertex['y'], vertex['z']]
    source_p0[i] = vertex['pressure_0']
    radius[i] = vertex['radius']

cloud_num = len(radius)

In [3]:
@ti.func
def caculate(a0, p0, xg, yg, zg, xc, yc, zc):
    R = ti.Vector([xg - xc, yg - yc, zg - zc]).norm() 
    intensity = p0 * fuck.step(0.0, a0  - R)
    return intensity

@ti.kernel
def rendering(
    grid_location: ti.types.ndarray(),
    cloud_location: ti.types.ndarray(),
    source_p0: ti.types.ndarray(),
    radius: ti.types.ndarray(),
    grid_intensity: ti.types.ndarray(),
):
    for idx in ti.ndrange(grid_points):
        xg = grid_location[idx, 0]*res
        yg = grid_location[idx, 1]*res
        zg = grid_location[idx, 2]*res
        for num in ti.ndrange(cloud_num):
            xc = cloud_location[num, 0]
            yc = cloud_location[num, 1]
            zc = cloud_location[num, 2]
            p0 = source_p0[num]
            a0 = radius[num]
            grid_intensity[idx] +=(
                caculate(0.5*a0, 10 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(0.6*a0, 9 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(0.9*a0, 8 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(1.2*a0, 7 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(1.5*a0, 6 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(1.8*a0, 5 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(2.1*a0, 4 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(2.4*a0, 3 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(2.7*a0, 2 * p0 / 55, xg, yg, zg, xc, yc, zc)
                + caculate(3.0*a0, 1 * p0 / 55, xg, yg, zg, xc, yc, zc)
            )

In [ ]:
import time

start = time.time()
rendering(
    grid_location,
    cloud_location,
    source_p0,
    radius,
    grid_intensity,
)
end = time.time()
print(f"Time cost: {end - start}s")


In [ ]:
grid_intensity_to_be_reshape = grid_intensity
print(grid_intensity_to_be_reshape.shape)

In [6]:
grid_intensity_reshaped = grid_intensity_to_be_reshape.reshape(200,520,400)

In [7]:
import scipy.io as sio
output_filename = 'SlingBAG_196_recon.mat'
sio.savemat(output_filename, {'SlingBAG_196_recon': grid_intensity_reshaped})